# Ablation study

## Setup

In [1]:
%useLatestDescriptors
%use dataframe, kandy

In [2]:
import java.nio.file.Path
import kotlin.io.path.*

fun <T> Path.readAllInstances(transform: (Path) -> DataFrame<T>) = toFile()
    .walk()
    .filter { it.isDirectory && (it.name.startsWith("f") || it.name.startsWith("v")) }
    .map {
        it.toPath()
            .listDirectoryEntries()
            .map(transform)
            .reduce { acc, dataFrame -> acc.concat(dataFrame) }
    }
    .reduce { acc, df -> acc.concat(df) }


fun readMetricsJson(json: Path): DataFrame<Int> {
    val metricsFile = json / "output/metrics.json"

    return when {
        metricsFile.exists() -> DataFrame.readJson(metricsFile.pathString).cast<Int>()
        else -> emptyDataFrame()
    }
}

fun <T> DataFrame<T>.groupByInstanceName() = groupBy {
    val instanceNames = column<String>("InputDir").map { it.removePrefix("study/instances/").split("/", limit = 2) }
    (instanceNames.map { it[0] } named "Severity") and (instanceNames.map {
        it[1].removePrefix("issue_").replace("_inlined", "")
    } named "Issue")
}

fun <T> DataFrame<T>.aggregateTotalAfterSize(toolName: String) = groupByInstanceName()
    .aggregate {
        get("Total").get("BeforeSize").cast<Int>().max() into "Before"
        get("Total").get("AfterSize").cast<Int>().mean() into toolName
    }

fun <T> DataFrame<T>.sortedByIssues() =
    sortWith { a, b ->
        when {
            a["Version"] == "final" -> 1
            b["Version"] == "final" -> -1
            else -> a["Version"].toString().compareTo(b["Version"].toString())
        }
    }
        .sortBy { get("Issue") }
        .sortByDesc { get("Severity") }
        .sortWith { dataRow, dataRow2 ->
            when {
                dataRow["Severity"] == "extra" -> 1
                dataRow2["Severity"] == "extra" -> -1
                else -> 0
            }
        }

fun DataFrame<*>.toLatexTable(): String {
    val header = columns().joinToString(" & ") { it.name() }
    val rows = rows().joinToString(" \\\\\n") { row ->
        row.values().joinToString(" & ") { when (it) {
            is Double -> "%.2f".format(it)
            null -> "-"
            else -> it.toString()
        } }
    }

    return """
        \begin{table}[h!]
            \centering
            \begin{tabular}{${"|c".repeat(columnsCount())}|}
                \hline
                $header \\
                \hline
                $rows \\
                \hline
            \end{tabular}
            \caption{todo}
            \label{tab:todo}
        \end{table}
    """.trimIndent()
}

In [3]:
val seruPerses = Path("results_perses").readAllInstances(::readMetricsJson)
val baseSeruSize = seruPerses.aggregateTotalAfterSize("All")
//baseSeruSize

In [74]:
fun <T> DataFrame<T>.aggregateCueTokensGT() = groupBy {
    val issueName = expr { get("issue").toString().removePrefix("issue_").removeSuffix("_inlined") }
    get("severity") named "Severity" and (issueName named "Issue") and (get("version") named "Version")
}
    .aggregate {
        get("tokens").cast<Int>().mean() into "mean"
        get("tokens").cast<Int>().std() into "std"
    }
    .sortedByIssues()
    .merge("Issue", "Version").by("/").into("Issue")

val cueTokensPersesGT = DataFrame.readCSV("groundtruth_perses/groundtruth_perses_tokens_cue_parser.csv")
val persesMeanGTTokens = cueTokensPersesGT.aggregateCueTokensGT()
    .remove("std")
    .rename("mean").into("Perses")
persesMeanGTTokens

Severity,Issue,Perses
semantic,2218/v1,47.000000
semantic,2218/v2,32.000000
semantic,2218/final,32.000000
panic,2490/v1,152.000000
panic,2490/v2,49.000000
panic,2490/final,44.000000
panic,2584/v1,53.400000
panic,2584/v2,48.000000
panic,2584/final,25.000000
error,2209/v1,145.200000


In [5]:
val strategies = listOf(
    "Let",
    "Empty Declaration",
    "Package",
    "Redundant Nesting",
    "List",
    "Trivial If",
    "If",
    "Ellipsis",
    "Constant Propagation",
    "String Interpolation",
    "Loop Unrolling",
    "Unification",
    "Union",
    "Import",
)

val strategyDFs = List(14) {
    Path("ablation_results", "strategy_$it").readAllInstances(::readMetricsJson)
}
val noConstantPropagation = Path("ablation_results", "no_constant_propagation").readAllInstances(::readMetricsJson)

In [6]:
val strategiesTotalSize = strategyDFs
    .mapIndexed { i, it -> it.aggregateTotalAfterSize(strategies[i]) }
    .reduce { acc, frame -> acc.fullJoin(frame) }

val noConstantPropagationTotalSize = noConstantPropagation.aggregateTotalAfterSize("All \\ Constant Propagation")

//strategiesTotalSize

# Difference to ground truth

In [73]:
val diffToBaseSize = persesMeanGTTokens
    .fullJoin(baseSeruSize)
    .fullJoin(noConstantPropagationTotalSize)
    .fullJoin(strategiesTotalSize)
    .move("Before").to(2)
    .update { allAfter("Perses") }.with {
        (it as Double) - getValue<Double>("Perses")
    }

val diffToBaseSizeRelative = diffToBaseSize
    .update { allAfter("Perses") }.with {
    ((it as Double) / getValue<Int>("Before")) * 100
}

diffToBaseSize

Severity,Issue,Before,Perses,All,All \ Constant Propagation,Let,Empty Declaration,Package,Redundant Nesting,List,Trivial If,If,Ellipsis,Constant Propagation,String Interpolation,Loop Unrolling,Unification,Union,Import
semantic,2218/v1,63,47.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
semantic,2218/v2,39,32.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
semantic,2218/final,37,32.000000,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
panic,2490/v1,420,152.000000,-67.000000,-44.000000,-31.000000,-7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-7.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
panic,2490/v2,67,49.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
panic,2490/final,48,44.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
panic,2584/v1,251,53.400000,-11.200000,-11.800000,-0.400000,-11.400000,0.400000,0.000000,-1.000000,-0.400000,0.400000,-1.400000,-11.400000,-0.400000,-0.400000,-0.400000,-0.400000,-0.400000
panic,2584/v2,106,48.000000,-19.000000,-6.000000,0.000000,-6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
panic,2584/final,25,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
error,2209/v1,274,145.200000,-11.200000,-12.200000,0.000000,-12.000000,-3.600000,-5.400000,-3.600000,-3.000000,-3.600000,-5.200000,-7.000000,-1.800000,0.200000,-3.600000,0.000000,1.800000


In [70]:
fun <T> DataFrame<T>.plotDifferenceToGroundTruth(yLabel: String, medianLabelOffset: Pair<Int, Int> = 4 to 6) =
    remove("Perses", "Before")
        .reorder { allAfter("Issue") }.by { it.cast<Double>().median() }
        .gather { allAfter("Issue") }.into("Heuristic", "Diff")
        .plot {
            line {
                x("Heuristic")
                y { constant(0) }
            }

            var middleStat: List<Double> = emptyList()
            boxplot("Heuristic", "Diff") {
                boxes {
                    alpha = .1
                    width = .5
                    middleStat = Stat.middle.toList()
                }
                outliers {
                    show = false
                }

            }

            text {
                x(get("Heuristic").toSet())
                y(List(16) { if (it % 2 == 0) medianLabelOffset.first else medianLabelOffset.second })
                label(middleStat.map { "%.2f".format(it) })
            }

            points {
                x("Heuristic")
                y("Diff")
                color("Heuristic") {
                    scale = categorical(
                        "All" to Color.hex("#1f77b4"), // Blue
                        "All \\ Constant Propagation" to Color.hex("#ff7f0e"), // Orange
                        "Constant Propagation" to Color.hex("#d62728"), // Red
                        "Empty Declaration" to Color.hex("#2ca02c"), // Green
                        "Let" to Color.hex("#9467bd"), // Purple
                        "Trivial If" to Color.hex("#8c564b"), // Brown
                        "Ellipsis" to Color.hex("#e377c2"), // Pink
                        "Redundant Nesting" to Color.hex("#7f7f7f"), // Gray
                        "List" to Color.hex("#bcbd22"), // Olive
                        "Package" to Color.hex("#17becf"), // Cyan
                        "Unification" to Color.hex("#4b0082"), // Indigo
                        "String Interpolation" to Color.hex("#6e5596"), // Dark Purple
                        "If" to Color.hex("#ff9896"), // Light Red
                        "Union" to Color.hex("#98df8a"), // Light Green
                        "Loop Unrolling" to Color.hex("#c5b0d5"), // Lavender
                        "Import" to Color.hex("#ffbb78")  // Peach
                    )
                }
                alpha = .7
                size = 2.5
                position = Position.jitter(.2)
            }

            layout {
                style {
                    xAxis {
                        text { blank = true }
                        title { blank = true }
                    }
                    yAxisLabel = yLabel
                }
                size = 1000 to 450
            }
        }

In [71]:
diffToBaseSize.plotDifferenceToGroundTruth("Difference to Perses [Token]")

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Dv37fk"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"Diff":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-67.0,-7.0,-44.0,-7.0,-31.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-5.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-11.199999999999996,-11.399999999999999,-11.799999999999997,-11.399999999999999,-0.3999999999999986,0.3999999999999986,0.0,-1.0,-0.3999999999999986,0.3999999999999986,-1.3999999999999986,-0.3999999999999986,-0.3999999999999986,-0.3999999999999986,-0.3999999999999986,-0.3999999999999986,-19.0,-19.0,-6.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-11.199999999999989,-7.0,-12.199999999999989,-12.0,0.0,-3.5999999999999943,-5.399999999999977,-3.5999999999999943,-3.0,-3.5999999999999943,-5.199999999999989,-1.799999999999983,0.20000000000001705,-3.5999999999999943,0.0,1.8000000000000114,-17.0,-14.0,-17.0,-2.0,0.0,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,1.2000000000000028,0.0,0.0,1.2000000000000028,1.2000000000000028,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-10.0,-2.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52.0,0.0,-28.0,-5.0,-11.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0],
"Heuristic":["All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagat

In [72]:
diffToBaseSizeRelative.plotDifferenceToGroundTruth("Reduction rate difference to Perses [%]", medianLabelOffset = 2 to 3)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="g2jm8k"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"Diff":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.6923076923076925,-2.564102564102564,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-8.108108108108109,-2.7027027027027026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.95238095238095,-1.6666666666666667,-10.476190476190476,-1.6666666666666667,-7.380952380952381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.2380952380952381,-7.462686567164178,-5.970149253731343,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.166666666666666,-4.166666666666666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.462151394422309,-4.541832669322709,-4.701195219123505,-4.541832669322709,-0.15936254980079625,0.15936254980079625,0.0,-0.398406374501992,-0.15936254980079625,0.15936254980079625,-0.5577689243027882,-0.15936254980079625,-0.15936254980079625,-0.15936254980079625,-0.15936254980079625,-0.15936254980079625,-17.92452830188679,-17.92452830188679,-5.660377358490567,-5.660377358490567,0.0,0.0,0.0,0.0,0.0,0.0,-0.9433962264150944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.087591240875908,-2.5547445255474455,-4.452554744525544,-4.37956204379562,0.0,-1.313868613138684,-1.9708029197080208,-1.313868613138684,-1.094890510948905,-1.313868613138684,-1.8978102189780979,-0.6569343065693368,0.07299270072993323,-1.313868613138684,0.0,0.6569343065693473,-13.709677419354838,-11.29032258064516,-13.709677419354838,-1.6129032258064515,0.0,0.0,0.0,0.0,-8.064516129032258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.9090909090909091,-0.9090909090909091,0.0,1.0909090909090935,0.0,0.0,1.0909090909090935,1.0909090909090935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-7.462686567164178,-14.925373134328357,-2.9850746268656714,0.0,0.0,-2.9850746268656714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.857142857142857,0.0,-2.857142857142857,-2.857142857142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-20.883534136546185,0.0,-11.244979919678714,-2.0080321285140563,-4.417670682730924,0.0,0.0,0.0,0.0,0.0,-0.8032128514056224,0.0,0.0,0.0,0.0,0.0],
"Heuristic":["All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Trivial If","If","Ellipsis","String Interpolation","Loop Unrolling","Unification","Union","Import","All","Constant Propagation","All \\ Constant Propagation","Empty Declaration","Let","Package","Redundant Nesting","List","Tr

# Experiments

In [81]:
val importDf = strategyDFs[13]
val packageDf = strategyDFs[2]
val trivialIfDf = strategyDFs[5]
val ifDf = strategyDFs[6]
val loopUnrollingDf = strategyDFs[10]
val unionDf = strategyDFs[12]
unionDf

<iframe onload="o_resize_iframe_out_32()" style="width:100%;" class="result_container" id="iframe_out_32" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
 white-space: nowrap;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_-738196442"></table>

<p class="dataframe_description">... showing only top 20 of 80 rows</p><p class="dataframe_description">DataFrame: rowsCount = 80, columnsCount = 6</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.le